In [1]:
pip install opencv-Python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/39.5 MB 8.4 MB/s eta 0:00:05
   --- ------------------------------------ 3.1/39.5 MB 8.8 MB/s eta 0:00:05
   ----- ---------------------------------- 5.2/39.5 MB 8.8 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/39.5 MB 8.7 MB/s eta 0:00:04
   --------- ------------------------------ 8.9/39.5 MB 8.8 MB/s eta 0:00:04
   ---------- ----------------------------- 10.7/39.5 MB 8.7 MB/s eta 0:00:04
   ------------- -------------------------- 12.8/39.5 MB 8.8 MB/s eta 0:00:04
   -------------- ------------------------- 14.7/39.5 MB 8.8 MB/s eta 0:00:03
   ---------------- ----------------------- 16.5/39.5 MB 8.7 MB/s eta 0:00:03
   ------------------ --------------------- 18.4/39.5 MB 8.8 MB/s eta 0:00:03
   -------------------- ------------------- 20.4/39.5 MB 8.8 MB/s eta 0:00:03
   ---------------------- ----------------- 22.3/39.5 MB 8.8 MB/s eta 0:00:02

In [2]:
print("hello")

hello


In [ ]:
import cv2
import numpy as np

# Step 1: Load the image
image_path = "Car.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Step 2: Resize the image (optional based on your requirements)
target_size = (640, 480)  # Resize to a fixed size
image = cv2.resize(image, target_size)

# Step 3: Convert to Grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 4: Apply Gaussian Blur to reduce noise
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Step 5: Edge Detection using Canny
edges = cv2.Canny(blurred_image, 100, 200)

# Step 6: Thresholding (optional)
_, threshold_image = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY)

# Step 7: Morphological Transformations
kernel = np.ones((5, 5), np.uint8)
dilated_image = cv2.dilate(threshold_image, kernel, iterations=1)

# Optional: Other Morphological Operations like Erosion
# eroded_image = cv2.erode(threshold_image, kernel, iterations=1)

# Step 8: Optional - Histogram Equalization to improve contrast
equalized_image = cv2.equalizeHist(gray_image)

# Displaying the images to visually inspect
cv2.imshow('Original Image', image)
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Edges', edges)
cv2.imshow('Threshold Image', threshold_image)
cv2.imshow('Dilated Image', dilated_image)
cv2.imshow('Equalized Image', equalized_image)

# Wait for a key press to close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()



In [1]:
import cv2
import numpy as np

# Initialize the video capture
video_path = "Drone.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Initialize the FAST detector and ORB descriptor
fast = cv2.FastFeatureDetector_create(threshold=25)  # FAST detector
orb = cv2.ORB_create()  # ORB detector and descriptor

# Parameters for the motion tracking (you can adjust this)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize variables for tracking keypoints across frames
prev_frame = None
prev_keypoints = None
prev_descriptors = None

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convert frame to grayscale (ORB and FAST work on grayscale images)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors on the current frame using ORB
    keypoints = fast.detect(gray_frame, None)
    keypoints, descriptors = orb.compute(gray_frame, keypoints)

    # Draw the keypoints on the current frame for visualization
    frame_with_keypoints = cv2.drawKeypoints(frame, keypoints, None, color=(0, 255, 0))

    if prev_frame is not None:
        # Use the good features to track (using Lucas-Kanade optical flow)
        prev_pts = np.array([kp.pt for kp in prev_keypoints], dtype=np.float32).reshape(-1, 1, 2)
        curr_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)

        # Flatten status array to 1D
        status = status.reshape(-1)

        # Filter the points based on the status (only keep points that were tracked successfully)
        good_new = curr_pts[status == 1]
        good_old = prev_pts[status == 1]

        # Draw lines and points to indicate the motion of the tracked features
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = old.ravel()  # Convert old points from 2D array to flat coordinates
            c, d = new.ravel()  # Convert new points from 2D array to flat coordinates
            frame_with_keypoints = cv2.line(frame_with_keypoints, (int(a), int(b)), (int(c), int(d)), (0, 0, 255), 2)
            frame_with_keypoints = cv2.circle(frame_with_keypoints, (int(c), int(d)), 5, (0, 0, 255), -1)

    # Show the frame with the keypoints and motion tracking
    cv2.imshow("Motion Tracking", frame_with_keypoints)

    # Update previous frame and keypoints for next iteration
    prev_frame = gray_frame
    prev_keypoints = keypoints
    prev_descriptors = descriptors

    # Break the loop on 'Esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the video capture and close all OpenCV windows 
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the MRI image (replace with your own MRI image file)
image_path = "Brain.jpg"  # Replace with your image file path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Step 1: Preprocessing - Remove noise using Gaussian Blur
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

# Step 2: Apply Histogram Equalization for contrast enhancement
equalized_image = cv2.equalizeHist(blurred_image)

# Step 3: Thresholding - Separate tumor from brain tissue
# Use Otsu's thresholding to automatically find a good threshold value
_, thresholded_image = cv2.threshold(equalized_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Step 4: Find contours to identify tumor regions
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 5: Filter small contours (you may adjust the area threshold as needed)
min_area = 500  # Minimum contour area to be considered a tumor
tumor_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

# Step 6: Draw the contours on the original image
output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert to BGR for visualization
cv2.drawContours(output_image, tumor_contours, -1, (0, 255, 0), 2)  # Draw green contours

# Step 7: Post-processing - Apply morphological operations to refine the result (optional)
kernel = np.ones((5, 5), np.uint8)  # Define a kernel for dilation
dilated_image = cv2.dilate(thresholded_image, kernel, iterations=2)  # Dilation to join small tumor parts

# Show the results
cv2.imshow("Original Image", image)
cv2.imshow("Preprocessed Image", equalized_image)
cv2.imshow("Thresholded Image", thresholded_image)
cv2.imshow("Tumor Contours", output_image)
cv2.imshow("Dilated Image", dilated_image)

# Wait for a key press and close the image windows
cv2.waitKey(0)
cv2.destroyAllWindows()
